# DataSet Import and Analysis

In this notebook, we are going to train an RNN for the task of sentiment analysis. Our dataset is the Yelp review data set. Let's download this data set and analyze it.

In [ ]:
!pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 15.0 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd

import contractions
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from tqdm import tqdm

import torch
from torch.utils.data import Dataset, DataLoader

from torch.nn.utils.rnn import pad_sequence

import torch.nn as nn

import torch.nn.functional as F
import torch
import torch.nn as nn

from sklearn.metrics import f1_score, roc_auc_score

from sklearn.metrics import classification_report

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
!gdown --id 1Ns456aZINhGcmnI_SCIHgT0q2x14jM2s

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1Ns456aZINhGcmnI_SCIHgT0q2x14jM2s
To: /content/yelp.csv
100% 8.09M/8.09M [00:00<00:00, 37.3MB/s]


In [ ]:
!gdown --id 1Xl92YAhgQVS2fR0MDUTJHcpN5_hxP67-

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1Xl92YAhgQVS2fR0MDUTJHcpN5_hxP67-
To: /content/test-gpt.csv
100% 24.6k/24.6k [00:00<00:00, 51.0MB/s]


In [ ]:
df = pd.read_csv('yelp.csv')
df.head()

,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,2,5,0
1,ZRJwVLyzEJq1VAihDhYiow,2011-07-27,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,review,0a2KyEL0d3Yb1V6aivbIuQ,0,0,0
2,6oRAC4uyJCsJl1X0WZpVSA,2012-06-14,IESLBzqUCLdSzSqm0eCSxQ,4,love the gyro plate. Rice is so good and I als...,review,0hT2KtfLiobPvh6cDC8JQg,0,1,0
3,_1QQZuf4zZOyFCvXc0o6Vg,2010-05-27,G-WvGaISbqqaMHlNnByodA,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",review,uZetl9T0NcROGOyFfughhg,1,2,0
4,6ozycU1RpktNG2-1BroVtw,2012-01-05,1uJFq2r5QfJG_6ExMRCaGw,5,General Manager Scott Petello is a good egg!!!...,review,vYmM4KTsC8ZfQBg-j5MWkw,0,0,0


In [ ]:
df.describe()

,stars,cool,useful,funny
count,10000.000000,10000.000000,10000.000000,10000.000000
mean,3.777500,0.876800,1.409300,0.701300
std,1.214636,2.067861,2.336647,1.907942
min,1.000000,0.000000,0.000000,0.000000
25%,3.000000,0.000000,0.000000,0.000000
50%,4.000000,0.000000,1.000000,0.000000
75%,5.000000,1.000000,2.000000,1.000000
max,5.000000,77.000000,76.000000,57.000000


In [ ]:
# Now let's see how many samples there exists in this data set
print("Total number of samples is: {}".format(len(df)))
print("Total number of 1 star reviews is: {} and the fraction is:{:0.2f}".format(len(df[df['stars']==1]), len(df[df['stars']==1])/len(df)))
print("Total number of 2 star reviews is: {} and the fraction is:{:0.2f}".format(len(df[df['stars']==2]), len(df[df['stars']==2])/len(df)))
print("Total number of 3 star reviews is: {} and the fraction is:{:0.2f}".format(len(df[df['stars']==3]), len(df[df['stars']==3])/len(df)))
print("Total number of 4 star reviews is: {} and the fraction is:{:0.2f}".format(len(df[df['stars']==4]), len(df[df['stars']==4])/len(df)))
print("Total number of 5 star reviews is: {} and the fraction is:{:0.2f}".format(len(df[df['stars']==5]), len(df[df['stars']==5])/len(df)))

Total number of samples is: 10000
Total number of 1 star reviews is: 749 and the fraction is:0.07
Total number of 2 star reviews is: 927 and the fraction is:0.09
Total number of 3 star reviews is: 1461 and the fraction is:0.15
Total number of 4 star reviews is: 3526 and the fraction is:0.35
Total number of 5 star reviews is: 3337 and the fraction is:0.33


In [ ]:
# We just need these rows: text, stars
df_new = df[['text', 'stars']].copy()
df_new.head()

,text,stars
0,My wife took me here on my birthday for breakf...,5
1,I have no idea why some people give bad review...,5
2,love the gyro plate. Rice is so good and I als...,4
3,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",5
4,General Manager Scott Petello is a good egg!!!...,5


In [ ]:
df_test = pd.read_csv('test-gpt.csv')
df_test.drop(columns='Unnamed: 0', inplace=True)
df_test.head()

,ratings,labels
0,The food was absolutely delicious! I couldn't ...,1
1,Excellent service and a charming ambiance. I h...,1
2,This restaurant has a fantastic selection of d...,1
3,The staff is incredibly attentive and friendly...,1
4,"I love how they use fresh, high-quality ingred...",1


# Text Preprocessing

Now, it's the time to apply text preprocessing. In text preprocessing we remove stopwords, remove punctuations, expand contractions, and make all words to lower.

In [ ]:
stop_words = set(stopwords.words('english'))

In [ ]:
print(stop_words)

{'to', 'we', 'all', "hadn't", 'no', 'are', 'weren', 'just', 'because', 'have', 'between', "should've", 'couldn', 'hasn', 'having', 'both', 'such', 'only', 'very', "doesn't", 'their', "isn't", 'shouldn', 'ma', 'under', 'doesn', 'them', 'other', 'down', 'same', 'll', 'being', 'for', 'yours', "aren't", 'your', 'doing', "shouldn't", "shan't", 't', 'is', 'if', 'been', 'once', "it's", 'an', 'y', 'on', 'shan', 're', "hasn't", 'which', 'with', 'herself', 'than', "mustn't", 'ours', 'the', "wasn't", 'that', 'be', 'where', 'any', 'too', "couldn't", 's', "she's", 'own', 'him', 'am', 'and', "wouldn't", "you're", 've', 'those', 'from', 'its', 'our', 'they', 'not', 'wouldn', 'whom', 'below', 'again', 'but', 'during', 'up', 'was', 'or', 'do', 'haven', 'into', 'what', 'at', 'm', 'it', 'were', 'her', 'd', 'myself', 'you', 'after', 'themselves', 'here', 'don', 'himself', 'isn', "didn't", 'needn', 'didn', 'out', 'aren', 'his', 'did', 'he', 'wasn', 'why', 'yourselves', "won't", 'then', "you've", "mightn't"

In [ ]:
def expand_sent(sent):
  expanded_words = [ contractions.fix(word) for word in sent.split()]
  expanded_sent = " ".join(expanded_words)
  return expanded_sent

In [ ]:
# Lets see an example
sent = "I'm not very satisfied. It isn't good!"
expand_sent(sent)

'I am not very satisfied. It is not good!'

In [ ]:
def clean_sent(sent):
  sent = expand_sent(sent)
  tokens = word_tokenize(sent)
  tokens = [word.lower() for word in tokens]
  tokens = [word for word in tokens if word.isalpha()]
  return " ". join(tokens)

In [ ]:
#Lets see an example
sent = "I'm not very satisfied. The price is 12$, it's not so cheep!!!???"
clean_sent(sent)

'i am not very satisfied the price is it is not so cheep'

In [ ]:
# Now lets clean the sentences in the data set
df_new['clean_text'] = df_new['text'].apply(clean_sent)
ratings={1:0, 2:0, 3:1, 4:1, 5:1}
df_new['new_stars'] = df_new['stars'].apply(lambda star: ratings[star])

In [ ]:
df_new

,text,stars,clean_text,new_stars
0,My wife took me here on my birthday for breakf...,5,my wife took me here on my birthday for breakf...,1
1,I have no idea why some people give bad review...,5,i have no idea why some people give bad review...,1
2,love the gyro plate. Rice is so good and I als...,4,love the gyro plate rice is so good and i also...,1
3,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",5,rosie dakota and i love chaparral dog park it ...,1
4,General Manager Scott Petello is a good egg!!!...,5,general manager scott petello is a good egg no...,1
...,...,...,...,...
9995,First visit...Had lunch here today - used my G...,3,first visit had lunch here today used my group...,1
9996,Should be called house of deliciousness!\n\nI ...,4,should be called house of deliciousness i coul...,1
9997,I recently visited Olive and Ivy for business ...,4,i recently visited olive and ivy for business ...,1
9998,My nephew just moved to Scottsdale recently so...,2,my nephew just moved to scottsdale recently so...,0


In [ ]:
tqdm.pandas()
df_test['clean_text'] = df_test['ratings'].apply(clean_sent)
df_test['new_stars'] = df_test['labels']
df_test

,ratings,labels,clean_text,new_stars
0,The food was absolutely delicious! I couldn't ...,1,the food was absolutely delicious i could not ...,1
1,Excellent service and a charming ambiance. I h...,1,excellent service and a charming ambiance i ha...,1
2,This restaurant has a fantastic selection of d...,1,this restaurant has a fantastic selection of d...,1
3,The staff is incredibly attentive and friendly...,1,the staff is incredibly attentive and friendly...,1
4,"I love how they use fresh, high-quality ingred...",1,i love how they use fresh ingredients the flav...,1
...,...,...,...,...
295,"The staff was unattentive, and it was hard to ...",0,the staff was unattentive and it was hard to g...,0
296,"The desserts were too sweet, and it was overwh...",0,the desserts were too sweet and it was overwhe...,0
297,The food lacked creativity and tasted like gen...,0,the food lacked creativity and tasted like gen...,0
298,"The restaurant had an insect problem, which wa...",0,the restaurant had an insect problem which was...,0


# ‌Build Vocabulary for the data set

In this section, we write a function to create a vocabulary for a given data set. To this aim, each word should be mapped to unique index.

In [ ]:
def build_vocab(df, min_freq):
  word2index ={}
  index2word = {}
  word_count ={}
  index2word[0] = '<PAD>'
  index2word[1] = '<UNK>'

  word2index['<PAD>'] = 0
  word2index['<UNK>'] = 1
  sentences = df['clean_text'].to_list()
  for sent in sentences:
    tokens = sent.split()
    for token in tokens:
      if  not (token in word_count):
        word_count[token] =1
      else:
        word_count[token]+=1
  index =2
  for word in word_count:
    if word_count[word]>=min_freq:
      index2word[index] = word
      word2index[word] = index
      index+=1
  return word2index, index2word, word_count

In [ ]:
from sklearn.model_selection import train_test_split
df_train, df_val = train_test_split(df_new, test_size=0.3, shuffle=True, random_state=123)

In [ ]:
#We have to build vocabulary based on the train sentences
min_freq = 4
word2index, index2word, word_count = build_vocab(df_train, min_freq)

In [ ]:
print(len(word2index))

8604


In [ ]:
print(index2word)

{0: '<PAD>', 1: '<UNK>', 2: 'my', 3: 'wife', 4: 'and', 5: 'i', 6: 'went', 7: 'here', 8: 'sat', 9: 'on', 10: 'the', 11: 'patio', 12: 'near', 13: 'canal', 14: 'temperature', 15: 'was', 16: 'perfect', 17: 'for', 18: 'being', 19: 'outside', 20: 'view', 21: 'of', 22: 'camelback', 23: 'sublime', 24: 'only', 25: 'complaint', 26: 'about', 27: 'seating', 28: 'it', 29: 'difficult', 30: 'to', 31: 'read', 32: 'our', 33: 'menus', 34: 'when', 35: 'we', 36: 'were', 37: 'there', 38: 'in', 39: 'evening', 40: 'waiter', 41: 'friendly', 42: 'helpful', 43: 'chicken', 44: 'salad', 45: 'sandwich', 46: 'had', 47: 'very', 48: 'good', 49: 'loved', 50: 'burger', 51: 'she', 52: 'ordered', 53: 'all', 54: 'a', 55: 'nice', 56: 'relaxing', 57: 'sunday', 58: 'dinner', 59: 'brunch', 60: 'at', 61: 'el', 62: 'today', 63: 'wonderful', 64: 'comfortable', 65: 'with', 66: 'umbrellas', 67: 'heaters', 68: 'service', 69: 'outstanding', 70: 'food', 71: 'delicious', 72: 'creme', 73: 'brulee', 74: 'french', 75: 'toast', 76: 'heave

In [ ]:
print(word_count)

{'my': 7729, 'wife': 253, 'and': 30582, 'i': 29087, 'went': 1065, 'here': 3172, 'sat': 247, 'on': 6542, 'the': 46676, 'patio': 344, 'near': 186, 'canal': 13, 'temperature': 22, 'was': 14478, 'perfect': 429, 'for': 10659, 'being': 653, 'outside': 416, 'view': 114, 'of': 14852, 'camelback': 58, 'sublime': 10, 'only': 1772, 'complaint': 76, 'about': 2364, 'seating': 210, 'it': 14491, 'difficult': 70, 'to': 21261, 'read': 177, 'our': 2356, 'menus': 61, 'when': 2507, 'we': 6556, 'were': 4290, 'there': 3866, 'in': 10276, 'evening': 180, 'waiter': 239, 'friendly': 1030, 'helpful': 268, 'chicken': 1173, 'salad': 887, 'sandwich': 616, 'had': 5170, 'very': 3111, 'good': 4761, 'loved': 365, 'burger': 582, 'she': 1368, 'ordered': 1198, 'all': 3193, 'a': 25052, 'nice': 1541, 'relaxing': 59, 'sunday': 189, 'dinner': 685, 'brunch': 93, 'at': 4551, 'el': 31, 'chorro': 1, 'today': 249, 'wonderful': 340, 'comfortable': 183, 'with': 7620, 'umbrellas': 4, 'heaters': 12, 'service': 2163, 'outstanding': 91,

Now, we have to write a mapping function that maps sentences to their indexes based on the vocabulary that we created.
We alse define a reversed mapping function that maps a sequence of indexes to the correponding words.

In [ ]:
def sentence_encoder (sent, word2index, max_length=100):
  sent = clean_sent(sent)
  tokens = sent.split()[:max_length+1]
  encoded_sent = []
  for word in tokens:
    if not (word in word2index):
      encoded_sent.append(0) # 0 is the index for <UNK> tag
    else:
      encoded_sent.append(word2index[word])
  return encoded_sent

In [ ]:
#lets see an example
sent ="Hello How are you ?"
sentence_encoder(sent, word2index)

[2111, 135, 139, 191]

In [ ]:
def sentence_decoder(sent_encoded, index2word):
  sent_decoded =[]
  for index in sent_encoded:
    sent_decoded.append(index2word[index])
  return " ".join(sent_decoded)

In [ ]:
#Let's see an example
sent_encoded =[1105, 304, 94, 22]
sentence_decoder(sent_encoded, index2word)

'party tired this camelback'

# Creating Pytorch Data Set


Now, we have to create a pytoch data set and also the data loaders

In [ ]:
class SentimentDataSet(Dataset):
  def __init__(self, df, word2index):
    super(SentimentDataSet, self).__init__()
    self.texts = df['clean_text']
    self.labels = df['new_stars']
    self.dic = word2index
  def __len__(self):
    return len(self.texts)
  def __getitem__(self, index):
    text = self.texts.iloc[index]
    label = self.labels.iloc[index]
    encoded_text = sentence_encoder(text, self.dic)
    return torch.tensor(encoded_text, dtype=torch.long), torch.tensor(label, dtype=torch.long)

In [ ]:
def collate_fn(batch):
  texts=[]
  labels =[]
  for item in batch:
    texts.append(item[0])
    labels.append(item[1])
  lens = [len(sent) for sent in texts]
  batched_texts = pad_sequence(texts, padding_value=word2index['<PAD>'], batch_first=True)
  return batched_texts, torch.tensor(labels, dtype=torch.long), lens

In [ ]:
batch_size = 32

In [ ]:
train_dataset = SentimentDataSet(df_train, word2index)
test_dataset = SentimentDataSet(df_test, word2index)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

train_len = len(train_dataset)
test_len = len(test_dataset)

train_eval_loader = DataLoader(train_dataset, batch_size = train_len, shuffle=False, collate_fn=collate_fn )
test_eval_loader = DataLoader(test_dataset, batch_size = test_len, shuffle=False, collate_fn=collate_fn )

In [ ]:
for texts, labels, lens in train_loader:
  print(texts.shape)
  print(labels.shape)
  print(lens)
  break

torch.Size([32, 101])
torch.Size([32])
[16, 34, 101, 35, 84, 101, 101, 101, 19, 89, 101, 89, 66, 101, 101, 101, 101, 101, 101, 101, 101, 7, 101, 101, 62, 101, 52, 101, 101, 101, 28, 101]


# Weighting Module

In [ ]:
class Weighting(nn.Module):
  def __init__(self, input_dim, hidden_dim, output_dim):
    super(Weighting, self).__init__()
    self.network = nn.Sequential(nn.Linear(input_dim, hidden_dim),
                                 nn.ReLU(),
                                 nn.Linear(hidden_dim, hidden_dim),
                                 nn.ReLU(),
                                 nn.Linear(hidden_dim, output_dim))
  def forward(self, x):

    x = self.network(x)
    x = torch.sigmoid(x)
    x = F.normalize(x, p=1, dim=0)

    return x

#The RNN Network

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
class RNNClassifier(nn.Module):
  def __init__(self, n_vocabs, embed_dim, hidden_dim, n_classes, n_layers, bidirectional=True):
    super(RNNClassifier, self).__init__()
    self.Embedding = nn.Embedding(n_vocabs, embed_dim)
    self.rnn = nn.LSTM(embed_dim, hidden_dim, batch_first=True,
                       bidirectional=bidirectional,num_layers=n_layers,
                       dropout = 0.3)

    self.fc = nn.Linear(2*hidden_dim if bidirectional else hidden_dim, n_classes)
    self.batchnorm = nn.BatchNorm1d(2*hidden_dim if bidirectional else hidden_dim)
    self.relu = nn.ReLU()
    self.drop = nn.Dropout(0.5)

  def forward(self, seq, lens):
    x = self.Embedding(seq)
    x= self.drop(x)
    rnn_out, (rnn_h, rnn_c) = self.rnn(x)
    x_h = torch.cat([rnn_out[i,len-1,:].unsqueeze(0) for i, len in enumerate(lens)], 0)


    x = self.batchnorm(x_h)
    x = self.relu(x)
    x= self.drop(x)

    x = self.fc(x)

    return F.log_softmax(x, dim=-1), x_h

In [ ]:
def encode_onehot_torch(num_classes, labels):
    y = torch.eye(num_classes).to(device)
    return y[labels]

In [ ]:
def loss_function_classifier(outputs, labels, weights, num_classes):
  onehot_labels = encode_onehot_torch(num_classes, labels)
  return torch.sum(- weights * (onehot_labels.float() * outputs).sum(1), -1)

In [ ]:
def loss_function_weights(outputs, labels, weights, num_classes, gamma):
  onehot_labels = encode_onehot_torch(num_classes, labels)
  return torch.sum(- weights * (onehot_labels.float() * outputs).sum(1), -1) - gamma*torch.sum(weights*torch.log(weights+1e-20))

In [ ]:
#Initializations
n_vocabs = len(word2index)
embed_dim = 16
hidden_dim = 32
n_classes = 2
n_layers = 1
bidirectional=False
model_classifier = RNNClassifier(n_vocabs, embed_dim, hidden_dim, n_classes, n_layers, bidirectional)
model_classifier = model_classifier.to(device)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [ ]:
w_inputdim = 2*hidden_dim if bidirectional else hidden_dim
w_hiddendim=32
weight_lr = 4e-3
model_weights = []
weight_optimizers=[]
for i in range(n_classes):
  model_weights.append(Weighting(w_inputdim, w_hiddendim, 1).to(device))
  weight_optimizers.append(torch.optim.Adam(model_weights[-1].parameters(), lr=weight_lr, ))

In [ ]:
print("number of model parameters:{}".format(sum(p.numel() for p in model_classifier.parameters() if p.requires_grad)))

number of model parameters:144194


# Define Loss Function and Optimizer for baseline model 1

In [ ]:
classifier_lr = 3e-3
classifier_optimizer = torch.optim.Adam(model_classifier.parameters(), lr=classifier_lr, )

# The Training and Test Loops for baseline model 1

In [ ]:
from tqdm import tqdm
epoch_classifier= 5
epoch_weights = 3
gamma=1.0

In [ ]:
def zero_grad_both():
  classifier_optimizer.zero_grad()
  for opt in weight_optimizers:
    opt.zero_grad()

In [ ]:
def run_classifier(texts, lens):
  output_probs, embds = model_classifier(texts, lens)
  return output_probs, embds

In [ ]:
def run_weights_generator(embds, labels):
  batch_size = embds.shape[0]
  max_label = int(labels.max().item() + 1)
  weights = torch.empty(batch_size).to(device)
  for i in range(max_label):
    labels_indices = (labels == i).nonzero().squeeze()
    if labels_indices is not None:
      sub_embds = embds[labels_indices,:]
      weight_ = model_weights[i](sub_embds)
      weights[labels_indices] = weight_.squeeze()
  return weights

In [ ]:
def train_one_epoch():
  model_classifier.train()
  for model in model_weights:
    model.train()

  for epoch_clf in range(epoch_classifier):
    print("Epoch {}/{} of classifier training".format(epoch_clf, epoch_classifier))
    loss_clfs = []
    acc_clfs = []
    for texts, labels, lens in tqdm(train_loader):
      texts, labels = texts.to(device), labels.to(device)
      zero_grad_both()
      output_probs, embds = run_classifier(texts, lens)
      weights1 = run_weights_generator(embds, labels)
      loss_clf = loss_function_classifier(output_probs, labels, weights1, n_classes)
      loss_clf.backward()
      loss_clfs.append(loss_clf.item())
      preds = torch.argmax(output_probs,dim=-1)
      acc_clfs.append(sum(preds==labels)/len(labels))
      classifier_optimizer.step()
    loss_clfs_avg = sum(loss_clfs)/len(loss_clfs)
    acc_clfs_avg = sum(acc_clfs)/len(acc_clfs)
    print("classifier loss:{:.3f}, classifier acc:{:.3f}".format(loss_clfs_avg,acc_clfs_avg ))
  for epoch_w in range(epoch_weights):
    print("Epoch {}/{} of Weight generator training".format(epoch_w, epoch_weights))
    loss_Ws =[]
    acc_Ws =[]
    for texts, labels, lens in tqdm(train_loader):
      texts, labels = texts.to(device), labels.to(device)
      zero_grad_both()
      output_probs, embds = run_classifier(texts, lens)
      weights2 = run_weights_generator(embds, labels)
      loss_W = -loss_function_weights(output_probs, labels, weights2, n_classes, gamma)
      loss_Ws.append(loss_W.item())
      preds = torch.argmax(output_probs, dim=-1)
      acc_Ws.append(sum(preds==labels)/len(labels))
      loss_W.backward()
      for opt in weight_optimizers:
        opt.step()
    loss_W_avg = sum(loss_Ws)/len(loss_Ws)
    acc_W_avg = sum(acc_Ws)/len(acc_Ws)

    print("weight generator loss:{:.3f}, classifier acc:{:.3f}".format(loss_W_avg,acc_W_avg ))

  return

In [ ]:
def class_f1(output, labels, type='micro', pos_label=None):
    # preds = output.max(1)[1].type_as(labels)
    preds = torch.argmax(output, -1)
    if pos_label is None:
        return f1_score(labels.cpu().numpy(), preds.cpu().numpy(), average=type)
    else:
        return f1_score(labels.cpu().numpy(), preds.cpu(), average=type, pos_label=pos_label)

In [ ]:
def evaluate_train():
  model_classifier.eval()
  print('evaluating train')
  with torch.inference_mode():
    for texts, labels, lens in tqdm(train_eval_loader):
      texts, labels = texts.to(device), labels.to(device)
      output_probs, _ = run_classifier(texts, lens)
      loss = F.nll_loss(output_probs, labels).item()
      preds = torch.argmax(output_probs, dim=-1)
      acc = (sum(preds==labels) / len(labels))
      f1_score = class_f1(output_probs, labels, type='macro')
      return loss, acc, f1_score

In [ ]:
def evaluate_test():
  model_classifier.eval()
  loss = 0
  acc = 0
  print("evaluating test")
  with torch.inference_mode():
    for texts, labels, lens in tqdm(test_eval_loader):
      texts, labels = texts.to(device), labels.to(device)
      output_probs, _ = run_classifier(texts, lens)
      loss = loss + F.nll_loss(output_probs, labels).item()
      preds = torch.argmax(output_probs, dim=-1)
      acc = acc + (sum(preds==labels) / len(labels))
      f1_score = class_f1(output_probs, labels, type='macro')
      return loss, acc, f1_score

In [ ]:
epochs = 20
for epoch in range(epochs):
    train_one_epoch()
    train_loss, train_acc, train_f1_score = evaluate_train()
    test_loss, test_acc, test_f1_score = evaluate_test()
    print("Major Results for Epoch{}/{}".format(epoch, epochs))
    print("Train loss:{:.3f}, Train acc:{:.3f}, Train f1_score:{:.3f}".format(train_loss, train_acc, train_f1_score))
    print("Test loss:{:.3f}, Test acc: {:.3f}, Test f1_score:{:.3f}".format(test_loss, test_acc, test_f1_score))
    print("*********************************************")
    print("*********************************************")
    print("*********************************************")

In [ ]:
complete_loader = DataLoader(test_dataset, batch_size = len(test_dataset), collate_fn=collate_fn)
target_names = ['Negative', 'Positive']
for texts, labels, lens in complete_loader:
  model_classifier.eval()
  texts, labels = texts.to(device), labels.to(device)
  logits, _ = model_classifier(texts, lens)
  preds = torch.argmax(logits, dim=-1)
labels, preds = labels.cpu().numpy(), preds.cpu().numpy()
print(classification_report(y_true=labels, y_pred=preds, target_names=target_names))

# Define Loss Function and Optimizer for baseline model 2

In [ ]:
class_weights = torch.tensor([3, 1.5 ]).to(device)
ce_loss = torch.nn.CrossEntropyLoss(weight= class_weights)
optimizer2 = torch.optim.Adam(model_classifier.parameters(), lr=classifier_lr, )

# The Training and Test Loops for baseline model 2

In [ ]:
def train_one_epoch(model, data_loader):
  model.train()

  batch_loss = []
  batch_accuracy =[]
  for texts, labels, lens in tqdm(data_loader):
    optimizer2.zero_grad()
    texts, labels = texts.to(device), labels.to(device)
    logits, _ = model(texts, lens)
    loss = ce_loss(logits, labels)
    batch_loss.append(loss)
    preds = torch.argmax(logits, dim=-1)
    batch_accuracy.append(sum(preds==labels)/len(labels))
    loss.backward()
    optimizer2.step()
  loss_avg = sum(batch_loss) / len(batch_loss)
  accuracy_avg = sum(batch_accuracy) / len(batch_accuracy)
  return loss_avg, accuracy_avg



In [ ]:
def test_one_epoch(model, data_loader):
  model.eval()

  batch_loss = []
  batch_accuracy = []
  with torch.no_grad():
    for texts, labels, lens in tqdm(data_loader):
      texts, labels = texts.to(device), labels.to(device)
      logits, _ = model(texts, lens)

      loss = ce_loss(logits, labels)
      batch_loss.append(loss)
      preds = torch.argmax(logits, dim=-1)
      batch_accuracy.append(sum(preds==labels)/len(labels))
    loss_avg = sum(batch_loss) / len(batch_loss)
    accuracy_avg = sum(batch_accuracy) / len(batch_accuracy)
  return loss_avg, accuracy_avg

In [ ]:
model2 = RNNClassifier(n_vocabs, embed_dim, hidden_dim, n_classes, n_layers, bidirectional)
model2 = model2.to(device)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [ ]:
epochs = 40
best_loss= 100
patience = 5
n_violated=0
for epoch in range(epochs):
  train_loss, train_acc = train_one_epoch(model2, train_loader)
  test_loss, test_acc = test_one_epoch(model2, test_loader)
  if test_loss < best_loss:
    best_loss = test_loss
    n_violated =0
  else:
    n_violated +=1
  print("Results for Epoch:{}".format(epoch))
  print("Train Loss:{:.3f}, Train Accuracy:{:.3f}".format(train_loss, train_acc))
  print("Test Loss:{:.3f}, Test Accuracy:{:.3f}".format(test_loss, test_acc))
  if n_violated > patience:
    break

100%|██████████| 10/10 [00:00<00:00, 112.30it/s]


Results for Epoch:0
Train Loss:0.687, Train Accuracy:0.639
Test Loss:0.691, Test Accuracy:0.584


100%|██████████| 10/10 [00:00<00:00, 107.24it/s]


Results for Epoch:1
Train Loss:0.698, Train Accuracy:0.633
Test Loss:0.692, Test Accuracy:0.590


100%|██████████| 10/10 [00:00<00:00, 118.47it/s]


...

Results for Epoch:14
Train Loss:0.695, Train Accuracy:0.637
Test Loss:0.693, Test Accuracy:0.573


100%|██████████| 10/10 [00:00<00:00, 75.04it/s]


Results for Epoch:15
Train Loss:0.696, Train Accuracy:0.634
Test Loss:0.691, Test Accuracy:0.589


100%|██████████| 10/10 [00:00<00:00, 87.93it/s]

Results for Epoch:16
Train Loss:0.689, Train Accuracy:0.640
Test Loss:0.690, Test Accuracy:0.576


In [ ]:
complete_loader = DataLoader(test_dataset, batch_size = len(test_dataset), collate_fn=collate_fn)
target_names = ['Negative', 'Positive']
for texts, labels, lens in complete_loader:
  model2.eval()
  texts, labels = texts.to(device), labels.to(device)
  logits, _ = model2(texts, lens)
  preds = torch.argmax(logits, dim=-1)
labels, preds = labels.cpu().numpy(), preds.cpu().numpy()
print(classification_report(y_true=labels, y_pred=preds, target_names=target_names))

              precision    recall  f1-score   support

    Negative       0.30      0.14      0.19       100
    Positive       0.66      0.83      0.74       200

    accuracy                           0.60       300
   macro avg       0.48      0.49      0.46       300
weighted avg       0.54      0.60      0.56       300



In [ ]:
complete_loader = DataLoader(train_dataset, batch_size = len(train_dataset), collate_fn=collate_fn)
target_names = ['Negative', 'Positive']
for texts, labels, lens in complete_loader:
  model2.eval()
  texts, labels = texts.to(device), labels.to(device)
  logits, _ = model2(texts, lens)
  preds = torch.argmax(logits, dim=-1)
labels, preds = labels.cpu().numpy(), preds.cpu().numpy()
print(classification_report(y_true=labels, y_pred=preds, target_names=target_names))

              precision    recall  f1-score   support

    Negative       0.15      0.10      0.12      1162
    Positive       0.83      0.88      0.86      5838

    accuracy                           0.75      7000
   macro avg       0.49      0.49      0.49      7000
weighted avg       0.72      0.75      0.73      7000

